# Load Package

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.file_processing.processing_predictions import load_pi_df_dict
from src.evaluation.perf_eval import get_prediction_performance_table, display_pred_perf
from src.evaluation.ue_eval import get_ue_performance_table, display_ue_perf, restructure_ue_df
from src.evaluation.pi_eval import get_pi_performance_table, display_pi_perf
from src.evaluation.pi_plots import plot_predictions_with_pi_across_methods
from src.misc import create_folder
from ue_pi_dicts import ue_dict, pi_dict, pi_order
from seed_file import seed

seed = 2023
data_label = "mimic"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_pi_predictions = join(fp_project_checkpoints, "pi_predictions")
fp_evaluation = join(fp_project_checkpoints, "model_evaluation")

# Seed filepaths
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)
fp_cur_pi_predictions_folder = join(fp_pi_predictions, str(seed))
create_folder(fp_cur_pi_predictions_folder)
fp_cur_evaluation_folder = join(fp_evaluation, str(seed))
create_folder(fp_cur_evaluation_folder)

# Load PI Prediction Dict

In [ ]:
pi_df_dict = load_pi_df_dict(fp_cur_pi_predictions_folder)

# Prediction Performance

In [ ]:
pred_perf_df = get_prediction_performance_table(pi_df_dict, ue_dict)
display_pred_perf(pred_perf_df)
pred_perf_df.to_csv(join(fp_cur_evaluation_folder, "pred_perf.csv"))

# UE Performance

In [ ]:
ue_perf_df = get_ue_performance_table(pi_df_dict, ue_dict)
ue_perf_df = restructure_ue_df(ue_perf_df)
display_ue_perf(ue_perf_df)
ue_perf_df.to_csv(join(fp_cur_evaluation_folder, "ue_perf.csv"))

# PI Performance

In [ ]:
pi_perf_df = get_pi_performance_table(pi_df_dict, pi_dict)
display_pi_perf(pi_perf_df)
pi_perf_df.to_csv(join(fp_cur_evaluation_folder, "pi_perf.csv"))

# Plot PI

In [ ]:
plot_predictions_with_pi_across_methods(
    pi_df_dict, pi_dict, record='465n', record_col="record", time_col="target_index",
    fp_cur_evaluation_folder=fp_cur_evaluation_folder, dpi=300, save_fig=True,
    display_feature="ABPsys (mmHg)", regressor_label="t+1", ylim=[70, 160],
    pi_order=["RUE Gauss Copula", "RUE KNN", "RUE CP"], xlabel="Minutes", ylabel="Systolic ABP",
    #xlim=[0,80]
) # ABPsys (mmHg), RESP (bpm), # ylim=[0, 50]